# Homework 1 - Berkeley STAT 157

Handout 1/22/2017, due 1/29/2017 by 4pm in Git by committing to your repository. Please ensure that you add the TA Git account to your repository.

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.
1. To be safe, print the notebook as PDF and add it to the repository, too. Your repository should contain two files: ``homework1.ipynb`` and ``homework1.pdf``. 

The TA will return the corrected and annotated homework back to you via Git (please give `rythei` access to your repository).

In [1]:
from mxnet import ndarray as nd
import numpy as np
import time

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use `wait_to_read()` on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see http://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.NDArray.wait_to_read.html for details. 

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

In [2]:
# 1. Construct two matrices A and B with Gaussian random entries of size  4096×4096. 
A = nd.array(nd.random.normal(shape=(4096,4096)))
B = nd.array(nd.random.normal(shape=(4096,4096)))
print("A is ", A)
print("B is ", B)
# 2. Compute C=AB using matrix-matrix operations and report the time.
tic = time.time()
C = nd.dot(A,B)
C.wait_to_read()
print("C is ", C)
print("Time taken to compute C = AB using matrix-matrix operations is ",time.time() - tic)
# 3. Compute C=AB, treating A as a matrix but computing the result for each column 
# of B one at a time. Report the time.
tic = time.time()
for i in range(4096):
    C[:,i] = nd.dot(A, B[:,i])
C.wait_to_read()
print("C is ", C)
print("Time taken to compute C = AB using a series of matrix-vector operations is ",time.time() - tic)
# 4. Compute C=AB, treating A and B as collections of vectors. Report the time.
tic = time.time()
C = nd.zeros((4096,4096))
for i in range(4096):
    A_ = A[:,i].reshape(4096,1)
    B_ = B[i,:].reshape(1,4096)
    C += nd.dot(A_,B_)
C.wait_to_read()
print("C is ", C)
print("Time taken to compute C = AB using a series of matrix-vector operations is ",time.time()- tic)


A is  
[[ 1.1630787   0.4838046   0.29956347 ...  1.6892971   1.0433246
   1.4861063 ]
 [ 1.1784046  -0.90841913 -0.37429735 ...  1.8522056  -1.8105638
  -1.0253092 ]
 [ 0.01498137  0.12917037  1.0849217  ... -0.8227322   0.23276128
  -1.4382302 ]
 ...
 [ 0.89425045  0.04806393  0.06638991 ... -0.47666615 -0.8127311
   1.0233623 ]
 [ 0.47591528 -0.34719887  0.54473215 ...  0.03410461  0.05183558
  -0.20439206]
 [-0.03636989 -2.5792546  -1.0895499  ...  0.8099311  -1.8143338
   0.92715806]]
<NDArray 4096x4096 @cpu(0)>
B is  
[[ 0.90497434 -0.8291843   0.5853669  ... -1.7501848  -0.5421702
   0.9640367 ]
 [-0.3738427   1.5078996  -1.4877979  ... -2.4101987   0.8088441
   1.518381  ]
 [-0.67060167 -2.0757017   0.1026388  ... -1.1176999  -0.7563752
  -1.0046532 ]
 ...
 [-1.0118687   0.04218809  0.70467323 ...  0.02222571  0.61793566
  -0.9676132 ]
 [ 1.3284602  -0.93981117 -1.621257   ... -0.02329569 -1.4178524
   1.1862246 ]
 [ 1.3548605  -2.0296354   0.7279279  ...  0.43979615  0.0606415

##### 5. Running these again in GPU we see that....its lightning fast!

## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

1. A symmetrix matrix $B \in \mathbb{S}^{n}$ is said to be positive semidefinite (PSD) if the associated quadratic form is non-negative, i.e.,
$$ x^TBx \geq 0,  \forall x \in \mathbb{R}^{n}. \\  \\
Hence, \ x^TBx\ =  \ x^TADA^Tx \ = \ y  D  y^T \ = \sum_{i=1}^{n} \lambda_{i} y_{i}^2 \geq 0\ \  (change \ of \ variable \ y = A^Tx) \ $$ 
 
2. $B$, a real symmetrix matrix has a complete set of orthogonal eigenvectors for which the corresponding eigenvalues are all real numbers, which prevents us from having to work with complex field. Furthermore, $B$, a PSD matrix, is convex, which is a useful property for optimization. Using $ADA^T$ is more helpful when operations like products of B is involved.


## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See http://d2l.ai/chapter_deep-learning-computation/use-gpu.html for details.

## 4. NDArray and NumPy 

Your goal is to measure the speed penalty between MXNet Gluon and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in NDArray. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in NDArray for assignments and copy to NumPy at the end.

In [3]:
A = nd.array(nd.random.normal(shape=(4096,4096)))
B = nd.array(nd.random.normal(shape=(4096,4096)))
# 1. Compute ∥ABi∥2 one at a time and assign its outcome to ci directly.
start = time.time()
c = np.zeros((4096,1))
for i in range(4096):
    c[i] = nd.norm(nd.dot(A, B[:,i])).asscalar()
end = time.time()
print("Method 1 takes ", end - start)

# 2. Use an intermediate storage d in NDArray for assignments and copy to NumPy at the end.
start = time.time()
c = np.zeros((4096,1))
d = nd.zeros((4096,1))
for i in range(4096):
    d[i] = nd.norm(nd.dot(A, B[:,i])).asscalar()
c = d.asnumpy()
end = time.time()
print("Method 2 takes ", end - start)

Method 1 takes  14.870416164398193
Method 2 takes  16.87332510948181


## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [4]:
A = nd.array(nd.random.normal(shape=(4096,4096)))
B = nd.array(nd.random.normal(shape=(4096,4096)))
C = nd.array(nd.random.normal(shape=(4096,4096)))
C += nd.dot(A,B)
print("C is ", C)

C is  
[[-1.49538794e+01 -2.90343456e+01 -1.20786583e+02 ...  1.12601952e+02
  -3.49232483e+01 -1.09423615e+02]
 [ 3.03782139e+01  2.29860592e+01 -5.66329117e+01 ...  2.33377476e+01
   5.01700706e+01  1.37430887e+01]
 [-2.43770561e+01 -3.49905968e+01 -3.10774288e+01 ... -4.34055090e+00
  -1.08096857e+01  7.63302536e+01]
 ...
 [-1.50713013e+02  1.06534897e+02 -1.25168953e+02 ...  5.68572617e+01
   1.28955231e+02 -1.09834503e+02]
 [-4.46852951e+01 -2.38864880e+01 -6.48403692e+00 ...  1.19722977e+02
  -5.85413208e+01 -9.49381638e+01]
 [-8.27188416e+01 -5.92363691e+00  3.18313980e+01 ... -1.21761522e+01
  -1.21261887e+02 -3.75423431e-02]]
<NDArray 4096x4096 @cpu(0)>


## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [5]:
x = np.linspace(-10,10,201)
x = nd.array(x).reshape(201,1)
y = np.linspace(1,20,20)
y = nd.array(y)
A = x ** y
print("A is ", A)

A is  
[[-1.0000000e+01  1.0000000e+02 -1.0000000e+03 ...  9.9999998e+17
  -1.0000000e+19  1.0000000e+20]
 [-9.8999996e+00  9.8009995e+01 -9.7029889e+02 ...  8.3451318e+17
  -8.2616803e+18  8.1790629e+19]
 [-9.8000002e+00  9.6040001e+01 -9.4119208e+02 ...  6.9513558e+17
  -6.8123289e+18  6.6760824e+19]
 ...
 [ 9.8000002e+00  9.6040001e+01  9.4119208e+02 ...  6.9513558e+17
   6.8123289e+18  6.6760824e+19]
 [ 9.8999996e+00  9.8009995e+01  9.7029889e+02 ...  8.3451318e+17
   8.2616803e+18  8.1790629e+19]
 [ 1.0000000e+01  1.0000000e+02  1.0000000e+03 ...  9.9999998e+17
   1.0000000e+19  1.0000000e+20]]
<NDArray 201x20 @cpu(0)>
